In [1]:
import sklearn
from sklearn import tree
import pandas as pd
import numpy as np
import  matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from IPython.display import SVG
from graphviz import Source
from IPython.display import  display

In [3]:
from IPython.display import  HTML
style = "<style>svg{width:70% !important;height:70% !important;}</style>"
HTML(style)

In [4]:
titanic_data = pd.read_csv('D:\\КУРСЫ DATA SCIENCE\\МОИ ЗАДАЧКИ\\Titanic\\train.csv',  encoding='utf-8')

In [5]:
titanic_data.head( )

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
X = titanic_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket' , 'Cabin'], axis = 1)
y = titanic_data.Survived
# axis = 1 значает, что удалятся столбцы, а не индексы датафрейма 
                      # Survived убрали, потому что будем ее предсказывать. ( Удаляем ненужные для предикшена столбцы)

In [7]:
X = pd.get_dummies (X) # конвертирует переменные (строчные) в числовые (0,1) автоматически

In [8]:
X = X.fillna({'Age' : X.Age.median( )}) # заменяет все пропущенные значения в возрасте на средний возраст пассажиров(медианный)

In [9]:
X.head( )

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,1


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size = 0.33, random_state=42) # 33% выборки из датафрейма пойдет на тестовую. 
# (42 - начальное значение рандомного генератора чисел)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
clf_rf = RandomForestClassifier()

In [14]:
parametrs = {'n_estimators': [10,20,30], 'max_depth': [2,5,7,10]}

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
grid_search_cv_clf = GridSearchCV(clf_rf, parametrs, cv=5)

In [19]:
grid_search_cv_clf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [2, 5, 7, 10],
                         'n_estimators': [10, 20, 30]})

In [20]:
grid_search_cv_clf.best_params_

{'max_depth': 5, 'n_estimators': 20}

Задача №1

Разминочная задачка - создайте модель RandomForestClassifier с 15 деревьями и максимальной глубиной равной 5 уровням, поместите её в переменную rf. Обучите модель на данных x_train и y_train, предскажите класс для наблюдений в x_test и поместите его в переменную predictions.

In [21]:
rf = RandomForestClassifier(n_estimators=15,max_depth = 5)

In [24]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=15)

In [25]:
predictions = rf.predict(X_test)

Продолжение предидущей задачи по рандом форесту:

In [26]:
best_clf = grid_search_cv_clf.best_estimator_

In [27]:
best_clf.score(X_test, y_test)

0.8101694915254237

In [29]:
feature_importances = best_clf.feature_importances_

In [30]:
feature_importances_df = pd.DataFrame({'features':list(X_train),
                                                                    'feature_importances': feature_importances})

In [34]:
feature_importances_df.sort_values('feature_importances', ascending=False) # Расположение по убыванию. Набор фичей показывает, по каким из них мы делали сплиты
#для максимального снижения энтрапии наших данных. Sex_male, Sex_female, Fare - фичи, по которым было главное распределение выживет пассажир или нет.

,features,feature_importances
6,Sex_male,0.262523
5,Sex_female,0.220963
4,Fare,0.168877
0,Pclass,0.148262
1,Age,0.080408
3,Parch,0.038010
2,SibSp,0.035752
7,Embarked_C,0.020484
9,Embarked_S,0.014433
8,Embarked_Q,0.010288
